In [1]:
%%bash 
export AWS_DEFAULT_PROFILE=diplomka && /home/perinja/aws/aws sso login --no-browser --profile AdministratorAccess

Browser will not be automatically opened.
Please visit the following URL:

https://device.sso.eu-north-1.amazonaws.com/

Then enter the code:

RZJC-TVQK

Alternatively, you may visit the following URL which will autofill the code upon loading:
https://device.sso.eu-north-1.amazonaws.com/?user_code=RZJC-TVQK
Successfully logged into Start URL: https://diplomka.awsapps.com/start


In [2]:
import zipfile
import time
import tempfile
import secrets
import random
import pandas as pd
import pandas as pd
import os
import multiprocessing
import json
import json
import json
import glob 
import boto3
import logging
import logging.handlers
from typing import Iterator, Optional
from getpass import getpass


from tqdm.notebook import tqdm
from tqdm import tqdm
from elasticsearch.helpers import scan
from elasticsearch import Elasticsearch
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor

In [15]:
boto3.setup_default_session(profile_name='AdministratorAccess')

In [16]:
s3 = boto3.resource('s3')
my_bucket = s3.Bucket('wmdwalm')

In [17]:
my_bucket.creation_date

datetime.datetime(2023, 4, 18, 10, 35, 15, tzinfo=tzlocal())

In [20]:
es = Elasticsearch(
    hosts=[{'host': 'atlas-kibana.mwt2.org', 'port':9200, 'scheme':'https'}],
    basic_auth=('perinja', getpass())
)
    
es.options(request_timeout=60)

if es.ping():
    print('connected to ES.')

 ········


connected to ES.


In [21]:
START = 1672531201000
END = 1680307201000

In [22]:
# https://github.com/sand-ci/AlarmsAndAlerts

def query_ps_trace(dt) -> Optional[Iterator[dict]]:
    query = {
        "query": {
            "bool": {"must": [
                {"range": {"timestamp": {"gt": dt[0], "lte": dt[1]}}}
            ]}
        }
    }
    try:
        return scan_gen(
            scan(
                es,
                index="ps_trace",
                query=query,
                filter_path=["_scroll_id", "_shards", "hits.hits._source"],
            )
        )
    except Exception as e:
        print(e)



def query_trace_change(dt) -> Optional[Iterator[dict]]:
    query = {
        "query": {
            "bool": {
                "must": [
                    {"term": {"event": "path changed"}},
                    {"range": {"created_at": {"gt": dt[0], "lte": dt[1]}}},
                ]
            }
        }
    }
    
    try:
        return scan_gen(
            scan(
                es,
                index="aaas_alarms",
                query=query,
                filter_path=["_scroll_id", "_shards", "hits.hits._source"],
            )
        )
    except Exception as e:
        print(e)


def query_specific_site(src_site, dest_site, start, end) -> Optional[Iterator[dict]]:
    query = {
        "query": {
            "bool": {"must": [
                        {"range": {"timestamp": {"gt": start, "lte": end}}},
                        {"term": {"dest_site": dest_site}},
                        {"term": {"src_site": src_site}},
            ]
                    }
        }
    }
    try:
        return scan_gen(
            scan(
                es,
                index="ps_trace",
                query=query,
                filter_path=["_scroll_id", "_shards", "hits.hits._source"],
            )
        )
    except Exception as e:
        print(e)

        
def scan_gen(scan) -> Iterator[dict]:
    while True:
        try:
            yield next(scan)["_source"]
        except Exception:
            break


## Traffic data

In [23]:
logger = logging.getLogger("batcher")

In [24]:
logger.setLevel("INFO")

In [25]:
fh = logging.handlers.RotatingFileHandler("log.txt", maxBytes=(1048576*5), backupCount=7)
fmt = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
fh.setFormatter(fmt)
logger.addHandler(fh)

In [26]:
def download_site_to_s3(src_site, dest_site, start, end):
    data: Optional[Iterator[dict]] = query_specific_site(src_site=src_site, dest_site=dest_site, start=start, end=end)
    iterate = True

    if data is None:
        print("No data")
        return
    
    with tempfile.NamedTemporaryFile(suffix='.zip') as file:
        logger.info("Downloading into %s", file.name)
        with tqdm() as files:
            with zipfile.ZipFile(file.name, mode="w", compression=zipfile.ZIP_DEFLATED, compresslevel=9) as zip_file: 
                while iterate:
                    try:
                        item = next(data)
                        ts = item["timestamp"]
                        file_name = datetime.fromtimestamp(ts/1000).isoformat() + f'-{secrets.token_hex(nbytes=6)}.json'
                        zip_file.writestr(file_name, json.dumps(item))
                        files.update()
                    except StopIteration as e:
                        logger.info("Testing %s", file.name)
                        zip_file.testzip()
                        zip_file.close()
                        iterate = False

            logger.info("Uploading %s", file.name)
            my_bucket.upload_file(file.name, f"sites/{src_site}-{dest_site}.zip")
            logger.info("OK %s", file.name)
    logger.info("Done")

In [27]:
download_site_to_s3(src_site="CA-SFU-T2",dest_site="CSCS-LCG2", start=START, end=END)

44635it [00:15, 2871.38it/s]
